In [15]:
import os
import cv2
import json
import random
from pathlib import Path
from sklearn.model_selection import train_test_split

def extract_frames_and_labels(video_path, traco_path, output_img_dir, output_lbl_dir, bbox_size=40):
    cap = cv2.VideoCapture(video_path)
    name = Path(video_path).stem
    frame_map = {}

    with open(traco_path, 'r') as f:
        data = json.load(f)
        for item in data['rois']:
            z = item['z']
            x, y = item['pos']
            frame_map.setdefault(z, []).append((x, y))  # 支持多个目标

    frame_idx = 0
    success, frame = cap.read()
    saved_frames = []

    while success:
        h, w = frame.shape[:2]
        frame_name = f"{name}_{frame_idx:04d}"
        img_path = output_img_dir / f"{frame_name}.jpg"
        label_path = output_lbl_dir / f"{frame_name}.txt"

        cv2.imwrite(str(img_path), frame)
        saved_frames.append(frame_name)

        with open(label_path, 'w') as f:
            if frame_idx in frame_map:
                for cx, cy in frame_map[frame_idx]:
                    bbox_w = bbox_h = bbox_size
                    x_center = cx / w
                    y_center = cy / h
                    w_norm = bbox_w / w
                    h_norm = bbox_h / h
                    f.write(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

        success, frame = cap.read()
        frame_idx += 1

    cap.release()
    return saved_frames


def split_and_copy(dataset_root, saved_frames, train_ratio=0.8):
    img_dir = dataset_root / "images"
    lbl_dir = dataset_root / "labels"

    train_imgs = dataset_root / "images" / "train"
    val_imgs   = dataset_root / "images" / "val"
    train_lbls = dataset_root / "labels" / "train"
    val_lbls   = dataset_root / "labels" / "val"

    for d in [train_imgs, val_imgs, train_lbls, val_lbls]:
        d.mkdir(parents=True, exist_ok=True)

    train_list, val_list = train_test_split(saved_frames, train_size=train_ratio, random_state=42)

    for fname in train_list:
        os.rename(img_dir / f"{fname}.jpg", train_imgs / f"{fname}.jpg")
        os.rename(lbl_dir / f"{fname}.txt", train_lbls / f"{fname}.txt")

    for fname in val_list:
        os.rename(img_dir / f"{fname}.jpg", val_imgs / f"{fname}.jpg")
        os.rename(lbl_dir / f"{fname}.txt", val_lbls / f"{fname}.txt")

    return train_list, val_list

def generate_data_yaml(dataset_root, yaml_path="data.yaml"):
    content = f"""train: {dataset_root}/images/train
val: {dataset_root}/images/val

nc: 1
names: ["hexbug"]
"""
    with open(yaml_path, 'w') as f:
        f.write(content)
    print(f"✅ 生成 data.yaml 到 {yaml_path}")

def build_dataset(videos_dir, traco_dir, dataset_root="hexbug_dataset", bbox_size=40):
    dataset_root = Path(dataset_root)
    img_dir = dataset_root / "images"  # 临时保存所有帧
    lbl_dir = dataset_root / "labels"
    img_dir.mkdir(parents=True, exist_ok=True)
    lbl_dir.mkdir(parents=True, exist_ok=True)

    all_frames = []

    for fname in sorted(os.listdir(videos_dir)):
        if fname.endswith(".mp4"):
            stem = Path(fname).stem
            video_path = os.path.join(videos_dir, fname)
            # 支持 .traco, .json, .traco.sec 三种后缀
            traco_path = None
            for ext in [".traco", ".json", ".traco.sec"]:
                candidate = os.path.join(traco_dir, stem + ext)
                if os.path.exists(candidate):
                   traco_path = candidate
                   break
            if traco_path is None:
               print(f"⚠️ 缺失标注文件: {stem}.traco/.json/.traco.sec")
               continue
 


            if not os.path.exists(traco_path):
                print(f"⚠️ 缺失 traco: {traco_path}")

                continue
            frames = extract_frames_and_labels(video_path, traco_path, img_dir, lbl_dir, bbox_size)
            all_frames.extend(frames)

    train_list, val_list = split_and_copy(dataset_root, all_frames)
    generate_data_yaml(dataset_root)
    print(f"✅ 数据集制作完成，训练样本: {len(train_list)}，验证样本: {len(val_list)}")

# === 调用 ===
build_dataset(
    videos_dir="training/training",    # 视频目录
    traco_dir="training/training",     # .traco文件目录
    dataset_root="hexbug_dataset",  # 输出根目录


    bbox_size=40                # 可调大小
)


✅ 生成 data.yaml 到 data.yaml
✅ 数据集制作完成，训练样本: 2659，验证样本: 665


In [20]:
from ultralytics import YOLO

# 加载 YOLOv8n 模型（nano 版本，速度快，适合小目标）
model = YOLO("yolov8n.pt")

# 开始训练
model.train(
    data="/home/hpc/tovl/tovl108v/hexbug_dataset/data.yaml",
    epochs=150,
    imgsz=768,
    batch=16,
    name="hexbug_yolov8n",
    lr0=0.001,
    patience=30,
    augment=True
)


Ultralytics 8.3.161 🚀 Python-3.12.10 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/hpc/tovl/tovl108v/hexbug_dataset/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=hexbug_yolov8n5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=30, per

train: Scanning /home/hpc/tovl/tovl108v/hexbug_dataset/labels/train... 2659 images, 0 backgrounds, 1 corrupt: 100%|██████████| 2659/2659 [00:02<00:00, 1151.15it/s]


train: /home/hpc/tovl/tovl108v/hexbug_dataset/images/train/training016_0079.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.000127]
train: New cache created: /home/hpc/tovl/tovl108v/hexbug_dataset/labels/train.cache
val: Fast image access ✅ (ping: 0.5±0.4 ms, read: 321.6±131.5 MB/s, size: 236.7 KB)


val: Scanning /home/hpc/tovl/tovl108v/hexbug_dataset/labels/val... 665 images, 0 backgrounds, 0 corrupt: 100%|██████████| 665/665 [00:00<00:00, 941.13it/s] 


val: New cache created: /home/hpc/tovl/tovl108v/hexbug_dataset/labels/val.cache
Plotting labels to runs/detect/hexbug_yolov8n5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 768 train, 768 val
Using 16 dataloader workers
Logging results to runs/detect/hexbug_yolov8n5
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        665       1442      0.928      0.914      0.932      0.517

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    135/150      4.16G      1.148     0.6025     0.8605          0        768: 100%|██████████| 167/167 [00:25<00:00,  6.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:05<00:00,  8.39it/s]


                   all        665       1442      0.929       0.92      0.931      0.516
EarlyStopping: Training stopped early as no improvement observed in last 30 epochs. Best results observed at epoch 105, best model saved as best.pt.
To update EarlyStopping(patience=30) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

135 epochs completed in 1.148 hours.
Optimizer stripped from runs/detect/hexbug_yolov8n5/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/hexbug_yolov8n5/weights/best.pt, 6.3MB

Validating runs/detect/hexbug_yolov8n5/weights/best.pt...
Ultralytics 8.3.162 🚀 Python-3.12.10 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
                                                        CUDA:1 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:08<00:00,  4.73it/s]


                   all        665       1442      0.898      0.893      0.929      0.525
Speed: 0.2ms preprocess, 9.0ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/hexbug_yolov8n5


In [ ]:
yolo task=detect mode=val model=runs/detect/hexbug_yolov8n5/weights/best.pt data=your_dataset.yaml
